In [2]:
import numpy as np
from scipy.stats import norm

##### Price the corresponding American call option with the same input as the previous problem using a 100 step CRR binomial tree. Also calculate delta, gamma, theta, vega and rho using the techniques discussed in the lecture. Present your answers to 4 decimal places.
###### Note: depending on the choice of software platforms and seeds for the random number generator the MC results may vary slightly (the differences will getting smaller as the # of simulated path increases). On the other hand for a given binomial tree construction (CRR tree with 100 steps in our case) the results on a tree are unambiguous.

In [6]:
def CRR(S0:float=100, K:float=100, T:float=1, r:float=0.06,
        q:float=0.06, sigma:float=0.35, dsigma:float=0.05, 
        dr:float=0.01, N:int=100)-> float:
    '''
    Implementation of CRR Binomial Tree to estimate the option 
    price. 

    @param S0: initial price
    @param K: strike price
    @param T: time to maturity
    @param r: risk free interest rate
    @param q: continuous dividend yield
    @param sigma: volatility
    @param N: timestep

    @return out: option price
    '''
    # Initialize the parameters
    dt = T / N
    u = np.exp(sigma * (dt**.5))
    d = 1/u
    a = np.exp((r-q)*dt)
    p = (a-d)/(u-d)
    disc = np.exp(-r*dt)

    # calculation of vega
    u_hat = np.exp((sigma+dsigma) * (dt**.5))
    d_hat = 1/u_hat
    p_hat = (a-d_hat)/(u_hat-d_hat)

    # calculation of rho
    a_hat2 = np.exp(((r+dr)-q)*dt)
    p_hat2 = (a_hat2-d)/(u-d)
    disc_hat2 = np.exp(-(r+dr)*dt)

    # initialize stock prices 
    S = np.zeros(N+1)
    S_hat = np.zeros(N+1)
    for i in range(len(S)):
        S[i] = S0 * u**i * d**(N-i)
        S_hat[i] = S0 * u_hat**i * d_hat**(N-i)
    
    # option payoff
    C = np.zeros(N+1)
    C_hat = np.zeros(N+1)
    C_hat2 = np.zeros(N+1)
    for i in range(len(C)):
        C[i] = max(S[i] - K, 0)
        C_hat[i] = max(S_hat[i] - K, 0)
        C_hat2[i] = max(S[i] - K, 0)
    
    # backtrace of the tree
    delta_lst = []
    gamma_lst = []
    for i in np.arange(N-1, -1, -1):
        for j in range(i+1):
            S = S0 * u**j * d**(i-j)
            S_hat = S0 * u_hat**j * d_hat**(i-j)

            C[j] = disc * (p*C[j+1] + (1-p)*C[j])
            C[j] = max(C[j], S-K)

            C_hat[j] = disc * (p_hat*C_hat[j+1] + (1-p_hat)*C_hat[j])
            C_hat[j] = max(C_hat[j], S_hat-K)
            C_hat2[j] = disc_hat2 * (p_hat2*C_hat2[j+1] + (1-p_hat2)*C_hat2[j])
            C_hat2[j] = max(C_hat2[j], S-K)
        
        # keep the value of 2nd and 3rd layer of the tree for delta, 
        # gamma, and theta calculation
        if i == 1:
            delta_lst.append(C[0])
            delta_lst.append(C[1])
        elif i == 2:
            gamma_lst.append(C[0])
            gamma_lst.append(C[1])
            gamma_lst.append(C[2])
    
    delta = (delta_lst[1]-delta_lst[0])/(S0*u-S0*d)
    h = 0.5*(S0*u**2-S0*d**2)
    gamma = ((gamma_lst[2]-gamma_lst[1])/(S0*u**2-S0) - \
             (gamma_lst[1]-gamma_lst[0])/(S0-S0*d**2))/h
    theta = (gamma_lst[1]-C[0])/(2*dt)
    vega = (C_hat[0]-C[0])/dsigma
    rho = (C_hat2[0]-C[0])/dr
    
    return {"option_price":C[0], 
            "delta": delta, 
            "gamma": gamma, 
            "theta": theta, 
            "vega": vega, 
            "rho": rho}

In [4]:
def BSM(s0:float=100, k:float=100, r:float=0.06, q:float=0.06, sigma:float=0.35, 
        T:float=1, call_option:bool=True) -> float:
    '''
    Implementation of black shore. Served as the true value of the option.
    @param s0: spot price for the stock
    @param k: strike price
    @param r: risk free rate
    @param q: dividend yield
    @param sigma: volatility
    @param T: time span
    @param call_option: if a call or put option
    @param seednum: random seed 
    
    @return out: value of the option 
    '''
    d1 = (np.log(s0/k) + (r - q + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma* np.sqrt(T)
    
    if call_option:
        out = s0 * np.exp(-q*T) * norm.cdf(d1) - k * np.exp(-r*T) * norm.cdf(d2)
    else:
        out = k * np.exp(-r*T) * norm.cdf(-d2) - s0 * np.exp(-q*T) * norm.cdf(-d1)
    return {"bsm": out}

In [7]:
CRR(), BSM()

({'option_price': 13.241257227890095,
  'delta': 0.5480841683241352,
  'gamma': 0.01117787442093923,
  'theta': -6.056389015633634,
  'vega': 37.37065521248901,
  'rho': 34.63368771806756},
 {'bsm': 13.083027408250246})